TODO:
~~Use numpy instead of 2d list~~
~~Clean up code~~
Finish enviorment and move code to seperate python file instead of juypter notebook (conflict errors with jupyter can be horrid)
Create Q-Learning agent (NO DeepL)
Create DeepL Agent (DQN)
Add experience replay
Adjust training values until satisfactory

FUTURE TODO(?)
Increase board size
Add different agent
Compete against human
Work with time instead of step

In [ ]:
# pip install -r requirements.txt

In [ ]:
# Imports
# pyright: ignore[reportMissingImports]

import os
# Keep using keras-2 (tf-keras) rather than keras-3 (keras).
os.environ['TF_USE_LEGACY_KERAS'] = '1'
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

from tf_agents.specs import array_spec
from tf_agents.specs import tensor_spec
from tf_agents.networks import network

from tf_agents.policies import py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import scripted_py_policy

from tf_agents.policies import tf_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import actor_policy
from tf_agents.policies import q_policy
from tf_agents.policies import greedy_policy

from tf_agents.trajectories import time_step as ts

In [ ]:
"""
Step Tetris Implementation:
Author: Nathan Delcampo
Date: 1/13/2026
Last Modifed: 2/3/2026
Python Version: 3.11.14

DESC: Tetris implementation with Numpy Array.
    Works off a steps with one action per step.
"""

import random
import sys

class Tetris:

    # Contains local coordinates of all tetris pieces at any given rotation
    tetris_pieces = {
        0: { # I
            0: [(0,0), (1,0), (2,0), (3,0)],
            90: [(1,0), (1,1), (1,2), (1,3)],
            180: [(3,0), (2,0), (1,0), (0,0)],
            270: [(1,3), (1,2), (1,1), (1,0)],
        },
        1: { # T
            0: [(1,0), (0,1), (1,1), (2,1)],
            90: [(0,1), (1,2), (1,1), (1,0)],
            180: [(1,2), (2,1), (1,1), (0,1)],
            270: [(2,1), (1,0), (1,1), (1,2)],
        },
        2: { # L
            0: [(1,0), (1,1), (1,2), (2,2)],
            90: [(0,1), (1,1), (2,1), (2,0)],
            180: [(1,2), (1,1), (1,0), (0,0)],
            270: [(2,1), (1,1), (0,1), (0,2)],
        },
        3: { # J
            0: [(1,0), (1,1), (1,2), (0,2)],
            90: [(0,1), (1,1), (2,1), (2,2)],
            180: [(1,2), (1,1), (1,0), (2,0)],
            270: [(2,1), (1,1), (0,1), (0,0)],
        },
        4: { # Z
            0: [(0,0), (1,0), (1,1), (2,1)],
            90: [(0,2), (0,1), (1,1), (1,0)],
            180: [(2,1), (1,1), (1,0), (0,0)],
            270: [(1,0), (1,1), (0,1), (0,2)],
        },
        5: { # S
            0: [(2,0), (1,0), (1,1), (0,1)],
            90: [(0,0), (0,1), (1,1), (1,2)],
            180: [(0,1), (1,1), (1,0), (2,0)],
            270: [(1,2), (1,1), (0,1), (0,0)],
        },
        6: { # O/Square
            0: [(1,0), (2,0), (1,1), (2,1)],
            90: [(1,0), (2,0), (1,1), (2,1)],
            180: [(1,0), (2,0), (1,1), (2,1)],
            270: [(1,0), (2,0), (1,1), (2,1)],
        }
    }
    
    # Initialze configs and reset board to start game
    def __init__(self, _rows=14, _cols=6, _render=False):
        self.config = {}
        self.config["render"] = _render
        self.config["rows"] = _rows
        self.config["cols"] = _cols
        self.reset()

    # Resets board to blank state
    def reset(self):
        # Get configs
        self.cols = self.config["cols"]
        self.rows = self.config["rows"]
        print(f"CURRENT CONFIG: {self.config}")

        # Setup board
        self.board = np.zeros((self.rows, self.cols))
        self.score = 0
        self.current_piece = random.randint(0,6)
        self.next_piece = random.randint(0,6) # Maybe change to bag system? (cycle through all pieces)
        self.pos = [0, int((self.config["cols"] / 2) - 2)] # Middle of top of board
        self.rotation = 0
        
        return self.board
    
    # TODO: Finish this
    # Condense information into input tensor and return 
    # (maybe have observation and action spec declared at start?)
    def _get_observation_spec(self):
        # observation spec should have predetermined size (board + next piece should never change)
        observation_spec = array_spec.BoundedArraySpec(shape=(self.rows * self.cols + 1), dtype=np.uint16, )

    # Clears all lines that are full
    def _clear_lines(self):

        # Get full lines
        lines_full = []
        for index, row in enumerate(self.board):
            if 0 not in row:
                lines_full.append(index)
                # print(index)
            
        
        # Delete full rows
        self.board = np.delete(self.board, lines_full, axis=0)
        
        # Insert blank rows at top
        new_lines = np.zeros((len(lines_full), self.cols), dtype=int)
        self.board = np.concat((new_lines, self.board), axis=0)

        # Give score based on lines cleared
        if (len(lines_full) == 4): # TETRIS (double points)
            self.score += 8
        else:
            self.score += len(lines_full)

    # Check current piece for collisions with sides and board
    def _is_colliding(self):

        # Get local coordinates and use with position to check for collision
        piece = self.tetris_pieces[self.current_piece][self.rotation]

        for x, y in piece:
            x += self.pos[0]
            y += self.pos[1]

            if x not in range(self.cols) \
            or y not in range(self.rows) \
            or self.board[y, x] == 1:
                return True
        return False 
            
    # Adds current piece to board, randomly select another piece, end game if colliding
    def _add_piece(self):

        # Get local coordinates and use with position to add to board
        piece = self.tetris_pieces[self.current_piece][self.rotation]

        for x, y in piece:
            x += self.pos[0]
            y += self.pos[1]

            self.board[y, x] = 1

        # Clear any full lines
        self._clear_lines()

        # Get next piece and reset position
        self.current_piece = self.next_piece
        self.next_piece = random.randint(0,6)
        self.pos = [0, int((self.cols / 2) - 2)]
        self.rotation = 0

        # End game if new piece is colloding
        if self._is_colliding():
            self._game_over()



    def _game_over(self):
        #TODO: Figure out scoring system
        # Theoretical scoring/reward system
        # 4 line clear 8 points
        # else lines cleared = points (3 lines = 3, etc)
        # death = -10 reward?

        #TODO: Figure out how to store scores
        # txt file probably?

        #with open("scores.txt", "w") as f:
        #    f.write(self.score)  
        #sys.exit()

        # Reset game
        self.reset()

    # Change rotation based on angle given
    def _rotate(self, angle):
        self.rotation += angle

        if self.rotation == 360:
            self.rotation = 0
        elif self.rotation < 0:
            self.rotation = 270

    # Main 'loop' function of game
    def step(self, action):

        # Do given action
        if (action == "COUNTERCLOCKWISE"):
            self._rotate(-90)
        elif(action == "CLOCKWISE"):
            self._rotate(90)
        elif(action == "RIGHT"):
            self.pos[0] += 1
        elif(action == "LEFT"):
            self.pos[0] -= 1

        # Check collisions and reverse action if not allowed
        if self._is_colliding():
            print("INVALID MOVE")
            # Reverse action
            if (action == "COUNTERCLOCKWISE"):
                self._rotate(90)
            elif(action == "CLOCKWISE"):
                self._rotate(-90)
            elif(action == "RIGHT"):
                self.pos[0] -= 1
            elif(action == "LEFT"):
                self.pos[0] += 1
        
        # Drop piece down by one and check collision, add to board if colliding
        self.pos[1] += 1
        if self._is_colliding():
            print("PIECE PLACED")
            self.pos[1] -= 1
            self._add_piece()

        # Render by printing if needed
        if (self.config["render"]):
            self._render()

        #TODO: Integrate with observation spec
        return self.board 
    
    # Creates deepcopy of current board state with current piece added
    def get_board_copy(self):
        tempboard = np.copy(self.board)

        for x, y in self.tetris_pieces[self.current_piece][self.rotation]:
            x += self.pos[0]
            y += self.pos[1]

            tempboard[y, x] = 2
        
        return tempboard

    # Render board state by printing by row
    def _render(self):
        # Add current piece to temp board
        tempboard = self.get_board_copy()

        for row in tempboard:
            print(row)
        print("")
        


In [ ]:
"""
Random Action Agent
Author: Nathan Delcampo
Date: 1/13/2026
Last Modifed: 1/27/2026
Python Version: 3.11.14

DESC: Random Action Agent that works off of tetris implementaiton
"""

import pygame

# Action_spec for 6 possible actions
# RIGHT LEFT COUNTERCLOCKWISE CLOCKWISE NOOP
action_spec = array_spec.BoundedArraySpec(
    shape=(),
    dtype=np.int32,
    minimum=0,
    maximum=4,
    name='action'
)

# Setup tensorflow for random actions
input_tensor_spec = tensor_spec.TensorSpec((2,), tf.float32)
time_step_spec = ts.time_step_spec(input_tensor_spec)

my_random_tf_policy = random_tf_policy.RandomTFPolicy(
    action_spec=action_spec, time_step_spec=time_step_spec)
observation = tf.ones(time_step_spec.observation.shape)
time_step = ts.restart(observation)

# Setup Pygame
RED = (255, 0, 0)
BLACK = (0, 0, 0)

tetris = Tetris(_render=False)
pygame.init()
screen = pygame.display.set_mode((300, 700))
pygame.event.clear()
running = True

# Main Loop
while running:

    # Wait for pygame event
    event = pygame.event.wait()

    # Only respond to key presses
    if event.type == pygame.quit:
        running = False
        pygame.quit()
        sys.exit()
    elif event.type == pygame.KEYDOWN:
        if event.key == pygame.K_ESCAPE:
            running = False
            pygame.quit()
            sys.exit()
        else:

            # Get action from random agent and apply
            action = my_random_tf_policy.action(time_step)
            match action[0]:
                case 0:
                    print("RIGHT")
                    tetris.step("RIGHT")
                case 1:
                    print("LEFT")
                    tetris.step("LEFT")
                case 2:
                    print("CC")
                    tetris.step("COUNTERCLOCKWISE")
                case 3:
                    print("C")
                    tetris.step("CLOCKWISE")
                case 4:
                    print("N")
                    tetris.step("NOOP")
            
            # Get board copy for rendering
            board = tetris.get_board_copy()
            
            # Draw screen
            screen.fill(BLACK)

            for row_index, row in enumerate(board):
                for col_index, tile_value in enumerate(row):
                    # Calculate the position for the current tile
                    x = col_index * 50
                    y = row_index * 50

                    if (tile_value) == 0:
                        color = BLACK
                    else:
                        color = RED

                    # Draw the rectangle
                    pygame.draw.rect(screen, color, (x, y, 50, 50))

            # Update the display
            pygame.display.flip()

In [ ]:
"""
Random Action Agent
Author: Nathan Delcampo
Date: 1/13/2026
Last Modifed: 1/27/2026
Python Version: 3.11.14

DESC: Random Action Agent that works off of tetris implementaiton
"""

tetris = Tetris(_render=True)
userinput = ""

while (userinput != "STOP"):
    userinput = input()

    match userinput:
        case "r":
            print("RIGHT")
            tetris.step("RIGHT")
        case "l":
            print("LEFT")
            tetris.step("LEFT")
        case "cc":
            print("CC")
            tetris.step("COUNTERCLOCKWISE")
        case "c":
            print("C")
            tetris.step("CLOCKWISE")
        case "n":
            print("N")
            tetris.step("NOOP")